In [99]:
from __future__ import print_function, division
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [100]:
year_list = [x for x in range(2016,1985,-1) if x != 1999]
print(year_list)

[2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986]


In [101]:
concat_list = []
for year in year_list:
    url = 'https://www.basketball-reference.com/allstar/NBA_{}_voting.html'
    response = requests.get(url.format(year))
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    text_all=''

    # scrapes html from website with player name and votes
    all = soup.find_all('div', id =re.compile('div_voting-results-'))

    # cleans data so player name and votes are separated by commas
    for i in range(2):
        text_all += all[i].text
        string = text_all.replace("\n"," ").replace("Frontcourt ","").replace("Backcourt ","").replace("    ","").strip()
        string = re.sub(r'\d+\.', '',string).strip()
        string = re.sub(r'\s+',' ',string)
        string = re.sub(r',','',string)
        string = re.sub(r'\s(\d*)\s', r',\1,',string)
        string = re.sub(r'([a-z])\s(\d)', r'\1,\2',string)

    #Takes out first comma, for 2012 and before
    string = re.sub(r'Guard |Center |Forward ', ',',string)
    if string[0] == ",":
        string = re.sub(r'Guard |Center |Forward ', ',',string)[1:]

    # makes a list with name and votes as elements
    all_list = string.split(",")

    # makes a list of list with name and vote grouped together. inserted year to first index
    new_list = []
    it = iter(all_list)
    for x in it:
        new_list.append([x, int(next(it))])
    for player_vote in new_list:
        player_vote.insert(0,year)

    df = pd.DataFrame(new_list, columns=['Year', 'Player','Votes'])
    concat_list.append(df)

In [102]:
df_concat = pd.concat(concat_list, axis=0)
df_concat.head()

,Year,Player,Votes
0,2016,LeBron James,1089206
1,2016,Paul George,711595
2,2016,Carmelo Anthony,567348
3,2016,Pau Gasol,566988
4,2016,Andre Drummond,515296


In [104]:
df_sort = df_concat.sort_values(['Year', 'Votes'], ascending=[False, False]).reset_index(drop=True)
df_sort.head()

,Year,Player,Votes
0,2016,Kobe Bryant,1891614
1,2016,Stephen Curry,1604325
2,2016,LeBron James,1089206
3,2016,Kevin Durant,980787
4,2016,Dwyane Wade,941466


In [105]:
list(df_sort)

['Year', 'Player', 'Votes']